In [2]:
import numpy as np
import sys
import os
import netCDF4 as nc
import bisect
import argparse
import math
import xarray as xr

In [3]:
def write_field(fname, data):
    """writes out data to a .bin file, based on MITgcm verification tests"""
    print 'wrote to file: '+fname
    if sys.byteorder == 'little':
        data.byteswap(True)
    fid = open(fname, "wb")
    data.tofile(fid)
    fid.close()

In [4]:
input_path = '/archive/net/CM2.6/regrid_annual/regrid_202/532545_surf_vel_C_gridded.nc'

In [9]:
# parser = argparse.ArgumentParser(
#         description='convert folder of daily surface C grid velocites to folder of .data  ')

# parser.add_argument('--input', metavar='input', 
#                     help='the input annual C grid file ')

# parser.add_argument('--out', type=str, metavar='out', 
#                     help='the output folder of .data of files')

# args = parser.parse_args()

vel_year = xr.open_dataset(input_path)
out_dir = '/home/net/test_output/'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    print "Output Directory:", out_dir

In [21]:
assert vel_year.usurf.isel(time=0).dims == ('yu_c', 'xu_c') #check lat x lon indexing
assert vel_year.vsurf.isel(time=0).dims == ('yv_c', 'xv_c') #check lat x lon indexing

for i in range(len(vel_year.time) - 364):
    out_filename_u = out_dir+'U.'+str(i).zfill(10)+'.data'
    out_filename_v = out_dir+'V.'+str(i).zfill(10)+'.data'
    u_data = vel_year.usurf.isel(time=i).values.astype(np.float32)
    v_data = vel_year.vsurf.isel(time=i).values.astype(np.float32)
    write_field(out_filename_u, u_data)
    write_field(out_filename_v, v_data) 

wrote to file: /home/net/test_output/U.0000000000.data
wrote to file: /home/net/test_output/V.0000000000.data


In [27]:
new_U = np.fromfile('/home/net/test_output/U.0000000000.data', dtype=np.float32).byteswap(True)
old_U = np.fromfile('/archive/net/CM2.6/regrid/regrid_2020_ro/U.0000000000.data', dtype=np.float32).byteswap(True)

In [32]:
new_V = np.fromfile('/home/net/test_output/V.0000000000.data', dtype=np.float32).byteswap(True)
old_V = np.fromfile('/archive/net/CM2.6/regrid/regrid_2020_ro/V.0000000000.data', dtype=np.float32).byteswap(True)

In [33]:
assert len(old_U) == len(new_U)
assert len(old_V) == len(new_V)

In [34]:
i = 0 
while i < 20: 
    j = np.random.randint(0, len(old_U))
    print '---------START------------'
    print 'INDEX = ', j 
    print 'OLD U ', old_U[j]
    print 'NEW U', new_U[j]
    print old_U[j] == new_U[j]
    
    print old_V[j] == new_V[j]
    print 'OLD V ', old_V[j]
    print 'NEW V', new_V[j]
    print '--------END---------------'
 
    
    i+= 1 

---------START------------
INDEX =  2721250
OLD U  0.0829188
NEW U 0.0828184
False
True
OLD V  0.0870087
NEW V 0.0870087
--------END---------------
---------START------------
INDEX =  5467658
OLD U  0.0
NEW U 0.0
True
True
OLD V  0.0
NEW V 0.0
--------END---------------
---------START------------
INDEX =  1321583
OLD U  0.00817108
NEW U 0.00817108
True
True
OLD V  0.022072
NEW V 0.022072
--------END---------------
---------START------------
INDEX =  6761948
OLD U  0.0327158
NEW U 0.0327617
False
True
OLD V  0.178221
NEW V 0.178221
--------END---------------
---------START------------
INDEX =  4280134
OLD U  0.0257231
NEW U 0.0257187
False
True
OLD V  0.236298
NEW V 0.236298
--------END---------------
---------START------------
INDEX =  1229925
OLD U  0.0267051
NEW U 0.0267051
True
True
OLD V  -0.00829884
NEW V -0.00829884
--------END---------------
---------START------------
INDEX =  3994171
OLD U  -0.133305
NEW U -0.133251
False
True
OLD V  -0.122238
NEW V -0.122238
--------END-------